In [1]:
import pandas as pd

## Filtering excel sheet

In [ ]:
df = pd.read_excel('./Gringotts.xlsx')
excel_file = pd.ExcelFile('./Gringotts.xlsx')

df['Numbers'] = df['College'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
df.drop(columns=['Numbers'], inplace=True)

In [29]:
import re
def extract_and_calculate_numbers(text):
  # numbers = re.findall(r'\d+\.\d+', text)
  numbers = re.findall(r'\d+(?:\.\d+)?', text)
  numbers = numbers[5:]
  if len(numbers) >= 5:
      tlr = float(numbers[0])
      rpc = float(numbers[1])
      go = float(numbers[2])
      oi = float(numbers[3])
      pr = float(numbers[4])
  else:
      tlr, rpc, go, oi, pr = None, None, None, None, None
  
  return pd.Series({'TLR': tlr, 'RPC': rpc, 'GO': go, 'OI': oi, 'PR': pr})


excel_file = pd.ExcelFile('./Gringotts.xlsx')
df = pd.read_excel(excel_file, sheet_name="2016")
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Name'].apply(extract_and_calculate_numbers)
df

# df['Numbers'] = df['Name'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
# df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
# df.drop(columns=['Numbers'], inplace=True)
df.to_excel('nirf/2016.xlsx', index=False)

In [2]:
df = pd.read_excel('./nirf/2021.xlsx')
df['Name'] = df['Name'].str.split('Close', expand=True)[0]
df.to_excel('nirf/2021.xlsx')

## Adding dummy data

In [2]:
subtopic_weights = {
    "TLR": {"SS": 20, "FSR": 30, "FQE": 20, "FRU": 30},
    "RPC": {"PU": 35, "QP": 40, "IPR": 15, "FPPP": 10},
    "GO": {"GPH": 40, "GUE": 15, "GMS": 25, "GPHD": 20},
    "OI": {"RD": 30, "WD": 30, "ESCS": 20, "PCS": 20},
}

In [31]:
df = pd.read_excel('./nirf/xl/2021.xlsx')
def add_dummy_data(df, property_name):
    subtopics = list(subtopic_weights[property_name].keys())
    total_value = df[property_name].values[0]

    for subtopic in subtopics:
        percentage = subtopic_weights[property_name][subtopic]
        dummy_value = (percentage / 100) * total_value
        df[subtopic] = dummy_value

for property in subtopic_weights.keys():
    add_dummy_data(df, property)

df.to_csv('nirf/2021.csv')

In [39]:
df = pd.read_csv('./nirf/2023.csv')
df.isna().any().any()

False

## Rank prediction

In [15]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
weightage = {
  'TLR': 0.30, 
  'RPC': 0.30, 
  'GO': 0.20, 
  'OI': 0.10, 
  'PR': 0.10
}

In [47]:
df = pd.read_csv('./nirf/2020.csv')
df = pd.read_csv("C:/Users/piyus/Downloads/archive/engineering.csv")

df['Weighted_Score'] = (
  df['tlr'] * weightage['TLR'] +
  df['rpc'] * weightage['RPC'] +
  df['go'] * weightage['GO'] +
  df['oi'] * weightage['OI'] +
  df['perception'] * weightage['PR']
)

df[34:]

,institute_id,name,tlr,rpc,go,oi,perception,city,state,rank,Weighted_Score
34,IR-E-U-0410,Malaviya National Institute of Technology,71.41,41.69,59.89,54.40,8.95,Jaipur,Rajasthan,35,52.243
35,IR-E-U-0098,Delhi Technological University,63.36,31.12,73.21,53.07,36.91,New Delhi,Delhi,36,51.984
36,IR-E-U-0476,Shanmugha Arts Science Technology Research A...,73.13,33.81,64.89,56.60,10.70,Thanjavur,Tamil Nadu,37,51.790
37,IR-E-U-0202,Birla Institute of Technology,70.07,34.81,58.75,51.57,30.48,Ranchi,Jharkhand,38,51.419
38,IR-E-U-0496,Aligarh Muslim University,70.58,45.83,56.57,40.89,10.70,Aligarh,Uttar Pradesh,39,51.396
...,...,...,...,...,...,...,...,...,...,...,...
195,IR-E-C-42227,Dr D Y Patil Institute of Technology,49.69,1.66,53.63,48.50,1.63,Pune,Maharashtra,196,31.144
196,IR-E-C-49660,Pimpri Chinchwad College of Engineering,46.11,7.36,50.73,48.76,0.55,Pune,Maharashtra,197,31.118
197,IR-E-C-34167,Ramrao Adik Institute of Technology,53.60,3.82,45.75,47.25,0.00,Navi Mumbai,Maharashtra,198,31.101
198,IR-E-C-48145,BVRIT Hyderabad,49.16,0.46,53.74,52.98,1.63,Hyderabad,Telangana,199,31.095


In [13]:
X = df[['Weighted_Score']]
y = df['rank']

In [21]:
model2 = RandomForestRegressor(n_estimators=100, random_state=0)
model1 = LinearRegression()
model1.fit(X, y)
model2.fit(X, y)

RandomForestRegressor(random_state=0)

In [51]:
df1 = pd.read_csv('./nirf/2023.csv')
scores = {'TLR': 30, 'RPC': 30, 'GO': 30, 'OI': 30, 'PR': 10}
scores = pd.DataFrame(scores, index=[0])
scores['Weighted_Score'] = (
  scores['TLR'] * weightage['TLR'] +
  scores['RPC'] * weightage['RPC'] +
  scores['GO'] * weightage['GO'] +
  scores['OI'] * weightage['OI'] +
  scores['PR'] * weightage['PR']
)
predictions = model1.predict(scores[['Weighted_Score']])
scores['Predicted_Rank1'] = predictions
# scores['Predicted_Rank1_Round'] = scores['Predicted_Rank1'].apply(round)

predictions = model2.predict(scores[['Weighted_Score']])
scores['Predicted_Rank2'] = predictions
# scores['Predicted_Rank2_Round'] = scores['Predicted_Rank2'].apply(round)
# df1 = df1.sort_values(by='Predicted_Rank')
# df1.reset_index(drop=True, inplace=True)
scores
# current_year_data = current_year_data.sort_values(by='Predicted_Rank')

rmse = np.sqrt(mean_squared_error(y, model2.predict(X)))
rmse

0.3574649394229694